# Nimisõnafraaside eraldamine

Kood, mis võtab sisendiks faili 'lausesisesed_anafoorid_kusimustega.xlsx' ja annab väljundiks faili 'lausesisesed_anafoorid_nimisonafraasidega.xlsx'. 

Väljundfaili tulpade tähendused:
* **Index** - lause järjekorra indeks failis,
* **Unnamed: 0** - lause indeks, mis on sama failis 'lausesisesed_anafoorid.xlsx' oleva indeksiga ja ühevõrra väiksem failis 'asesonad_lausetes_margendatud.xlsx' oleva indeksiga,
* **Label** - käsitsi lisatud märgend, mis näitab, kas anafoor on lausesisene ('1'), lauseväline ('2') või pole tegemist anafooriga ('0'). Samuti leidub failis ka märgendusega '-1' lauseid (s.t tegu on veaga/märgitud sõna pole asesõna) ja märgendusega '3', kus olenevalt lause tõlegndusest võib, anafoor olla nii lausesisene kui ka väline. Antud failis on ainult märgendiga '1' ehk lausesiseste anafooridega laused,
* **pronoun** - lauses leiduv asesõna,
* **pronoun lemma** - lauses leiduva asesõna lemma,
* **pronoun sentence** - lause, kus selles leiduv asesõna on ümbritsetud topelt tärnidega - \*\*<asesõna>\*\*,
* **question** - automaatselt koostatud küsimus, mis on moodustatud lauses märgitud asesõna kohta.
* **N_phrases** - lauses leiduvate nimisõnafraaside juured

In [1]:
# Vajalike teekide import
from estnltk import Text
from estnltk import download
from estnltk_neural.taggers import StanzaSyntaxTagger
from estnltk.taggers.standard.syntax.phrase_extraction.phrase_extractor import PhraseExtractor
import pandas as pd

C:\Users\riintoom\AppData\Local\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
def phrases(lause, extractor_1, extractor_2, extractor_3, extractor_4, extractor_5, extractor_6, extractor_7, extractor_8):
    lause = Text(lause).tag_layer()
    #uudis.tag_layer('morph_extended')
    stanza_tagger.tag(lause)
    extractor_1.tag(lause)
    extractor_2.tag(lause)
    extractor_3.tag(lause)
    extractor_4.tag(lause)
    extractor_5.tag(lause)
    extractor_6.tag(lause)
    extractor_7.tag(lause)
    extractor_8.tag(lause)
    return lause

In [3]:
# Laeb alla stanzasyntaxtaggeri
download('stanzasyntaxtagger')

Resource 'stanza_syntax_2023-01-21' has already been downloaded.


True

In [4]:
# Initsialiseerib stanza taggeri ja retaggeri
stanza_tagger = StanzaSyntaxTagger(input_type='morph_analysis', input_morph_layer='morph_analysis')


In [5]:
# Initsialiseerib vajalikud PhraseExtractorid
extractor_nusbj = PhraseExtractor(deprel='nsubj', output_layer='nsubj_phrases')
extractor_obl = PhraseExtractor(deprel='obl', output_layer='obl_phrases')
extractor_obj = PhraseExtractor(deprel='obj', output_layer='obj_phrases')
extractor_nmod = PhraseExtractor(deprel='nmod', output_layer='nmod_phrases')
extractor_conj = PhraseExtractor(deprel='conj', output_layer='conj_phrases')
extractor_root = PhraseExtractor(deprel='root', output_layer='root_phrases')
extractor_appos = PhraseExtractor(deprel='appos', output_layer='appos_phrases')
extractor_iobj = PhraseExtractor(deprel='iobj', output_layer='iobj_phrases')

In [6]:
# Loeb sisse andmed (lausesiseste anafooridega lausete tabel)
df = pd.read_excel('lausesisesed_anafoorid_kusimustega.xlsx')
laused = df["pronoun sentence"].tolist()

In [8]:
# Leiab igast lauses nimisõnafraaside juured
result = []

for lause in laused:
    lause = lause.replace("*", "")
    margendatud = phrases(lause, extractor_nusbj, extractor_obl, extractor_obj, extractor_nmod, extractor_conj, extractor_root, extractor_appos, extractor_iobj)
    nimisonafraasid = []
    
    for el in margendatud.nsubj_phrases:
        if el.root.upostag != 'P':
            nimisonafraasid.append(el.root.text)
        
    for el in margendatud.obl_phrases:
        if el.root.upostag != 'P':
            nimisonafraasid.append(el.root.text)
        
    for el in margendatud.obj_phrases:
        if el.root.upostag != 'P':
            nimisonafraasid.append(el.root.text)
        
    for el in margendatud.nmod_phrases:
        if el.root.upostag != 'P':
            nimisonafraasid.append(el.root.text)
        
    for el in margendatud.conj_phrases:
        if el.root.upostag != 'P':
            nimisonafraasid.append(el.root.text)
        
    for el in margendatud.root_phrases:
        if el.root.upostag != 'P':
            nimisonafraasid.append(el.root.text)
        
    for el in margendatud.appos_phrases:
        if el.root.upostag != 'P':
            nimisonafraasid.append(el.root.text)
        
    for el in margendatud.appos_phrases:
        if el.root.upostag != 'P':
            nimisonafraasid.append(el.root.text)
        
    for el in margendatud.iobj_phrases:
        if el.root.upostag != 'P':
            nimisonafraasid.append(el.root.text)
        
    nimisonafraasid = list(set(nimisonafraasid))
    
    result.append(nimisonafraasid)
        

In [9]:
# Lisab leitud nimisõnafraasid tabelisse
df.insert(8, "N_phrases", result, False)

In [10]:
# Eemaldab juurde tekkinud index rea
df = df.drop(columns=["Index"])

In [11]:
# Salvestab nimisõnafraasidega täiendatud tabeli faili 'lausesisesed_anafoorid_nimisonafraasidega.xlsx'
writer = pd.ExcelWriter("lausesisesed_anafoorid_nimisonafraasidega.xlsx", engine="xlsxwriter", engine_kwargs={"options": {"strings_to_formulas": False}},
)

with writer as writer:
    df.to_excel(writer)